In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
local_download_path = "/content/drive/MyDrive/24-25/Machine Learning/"
download_path = local_download_path + "vocadb_data.zip"
!cp "{download_path}" vocadb.zip
!unzip -qo vocadb.zip

In [3]:
import pandas as pd
import nltk
import csv
import os

In [4]:
ex_df = pd.read_csv('inputs/2005to2009.csv',sep="|")

# visual checks to make sure data is in correct format
print("Data Preview:")
print(ex_df)
print("Data overview:")
print(ex_df.info())

Data Preview:
          ID        Song Name                              Author Name  \
0     531653        RULE ～約束～                         トマ豆腐 feat. 神威がくぽ   
1     123445        君、空、紙飛行機。                            立秋 feat. 初音ミク   
2      66561         東ノ暁　西ノ黄昏             hinayukki feat. MEIKO, KAITO   
3      49619            つきこもり                        ●テラピコス feat. 初音ミク   
4       6083           シンメトリー                         Re:nG feat. 初音ミク   
...      ...              ...                                      ...   
2251  125019       メロディック妹メタル                     karimono feat. MEIKO   
2252  533756    True My Heart                       ave;new feat. 佐倉紗織   
2253  193997              you                             dai feat. 癒月   
2254  592884       おはよう！！朝ご飯           Hiroto Sasaki feat. Mayako Nigo   
2255  462705  Trick or Treat!  seiya-murai feat. Desmond, Robert, Andy   

             Publish Date                                     English Lyrics  \
0     2008-12-31 

In [5]:
data_pieces = []

for filename in os.listdir("/content/inputs"):
    if filename.endswith("csv"):
        song_df = pd.read_csv("inputs/" + filename, sep="|")
        data_pieces.append(song_df)

all_data = pd.concat(data_pieces)
print(all_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 66539 entries, 0 to 7738
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               66539 non-null  int64 
 1   Song Name        66538 non-null  object
 2   Author Name      66539 non-null  object
 3   Publish Date     66539 non-null  object
 4   English Lyrics   7791 non-null   object
 5   Japanese Lyrics  66539 non-null  object
 6   Rating           66539 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 4.1+ MB
None


In [15]:
!pip install faiss-cpu
import torch.nn.functional as F
import numpy as np

from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

In [10]:
en_lyrics = all_data["English Lyrics"].dropna().to_list()
jp_lyrics = all_data["Japanese Lyrics"].to_list()

model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')

pool = model.start_multi_process_pool()
en_emb = model.encode_multi_process(en_lyrics, pool)
model.stop_multi_process_pool(pool)

pool = model.start_multi_process_pool()
jp_emb = model.encode_multi_process(jp_lyrics, pool)
model.stop_multi_process_pool(pool)

In [29]:
import faiss

d = en_emb.shape[1]

nlist = 100
m = 8
k = 4
quantizer = faiss.IndexFlatL2(d)
en_index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)

en_index.train(en_emb)
en_index.add(en_emb)

jp_index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)

jp_index.train(jp_emb)
jp_index.add(jp_emb)

In [56]:
def get_song_from_id(id, use_en=False):
  if use_en:
    return en_lyrics[id]
  else:
    return jp_lyrics[id]

def find_similar_lyrics(song_name, k, use_en=False):
  song = all_data[all_data['Song Name'] == song_name]
  if song.empty:
    return []
  song_en_lyrics = song['English Lyrics']
  if not song_en_lyrics.empty:
    song_en_lyrics = song_en_lyrics.values[0]
  song_jp_lyrics = song['Japanese Lyrics']
  if not song_jp_lyrics.empty:
    song_jp_lyrics = song['Japanese Lyrics'].values[0]
  if (isinstance(song_jp_lyrics, str) and not song_jp_lyrics) and (isinstance(song_en_lyrics, str) and not song_en_lyrics or pd.isna(song_en_lyrics)):
    return []
  if use_en:
    en_ind = en_lyrics.index(song_en_lyrics)
    D, I = en_index.search(en_emb[en_ind].reshape(1, -1).astype(np.float32), k)
  else:
    jp_ind = jp_lyrics.index(song_jp_lyrics)
    D, I = jp_index.search(jp_emb[jp_ind].reshape(1, -1).astype(np.float32), k)
  all_reccs = []
  for item_list in I:
    for item in item_list:
      all_reccs.append(get_song_from_id(item, use_en))
  return all_reccs

In [58]:
print(find_similar_lyrics("ココロ", 5))
print(find_similar_lyrics("CIRCUS PANIC", 5))

['道端に置いてきた\r\n怒り 悲しみ 憎しみ\r\n道端で拾ってきた\r\n喜び 楽しみ その感情\r\n\r\nころころ変わる僕の心\r\nガラスのように砕けて\r\n散って 散って 散って\r\nその破片を拾い集めたら\r\n何ができるのだろうか\r\n\r\nさあさあどうしようか\r\nどうしようか\r\n散りばめられた破片は\r\n星のように舞う\r\n\r\nさあさあどうしようか\r\nどうしようか\r\nもう復元不可能？\r\n小さく聞こえる君の声\r\n\r\n砕けていたのは僕自身でした\r\n', '砕けた欠片集めても元には\r\nもう戻れはしない\r\n鮮やかで儚げに\r\n知らない誰かが投げた世界の終わりで\r\n伝わらない気持ち抱えて\r\n\r\n「その手で触れたら二つにも三つにも」\r\n幸せな嘘で満たされていく日々\r\nうまく思い出せない夢\r\n不安定な記憶\r\n溶けていく心に黒い影が差し込む\r\n\r\n※\r\n見せかけの固い表面は\r\nもろく崩れゆく中身ととなりあわせ\r\n割られ砕かれ儚く散りゆくだけ\r\nやるせのない気持ち抱えて\r\n\r\n不意に切り出された思い出と\r\n口の中で消えてゆくビスケット\r\n遠くに聞こえた飛行機の飛ぶ音に\r\n空振りの気持ち重ねた\r\n\r\n※繰り返し', 'ともされる灯りはいつも消えそうでも\r\nまぶしくもないくらい生き延びている\r\n向けられた怒りも\u3000塗り重ねた罪も\r\n思い出せなくなる\r\n\r\n寂れた世界に告げて\r\n終わりにしよう\u3000綺麗なままで\r\n足もとめられない\u3000軽薄に打ち捨てられている\r\n売れ残りの迷子\r\n\r\n凍えた声すらむなしく響いてる\r\nわずかに残る繋がりかけの愛\r\nかわされた言葉も\u3000そむけられた目も\r\nどうにでもよくなる\r\n\r\n見れない夢にすがる\r\n終わりにしよう\u3000知らないままでいい\r\n気にもとめられない\u3000路地裏にはねのけても\r\n知らないふりしたいだけなの\r\n\r\n寂れた世界に告げて\r\n終わりにしよう\u3000綺麗なままで\r\n足もとめられない\u3000軽薄に打ち捨てられている\r\n売れ残りの迷子', '気の向くま